In [4]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd


In [182]:
!pip install -U ipykernel

  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5
  Attempting uninstall: jupyter-core
    Found existing installation: jupyter-core 4.7.1
    Uninstalling jupyter-core-4.7.1:
      Successfully uninstalled jupyter-core-4.7.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.5
    Uninstalling ipykernel-5.5.5:
      Successfully uninstalled ipykernel-5.5.5


You should consider upgrading via the 'c:\Users\johns\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\johns\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
def get_token():
    client_id = '7727b6a6602f4dcf99d337ee11f0983e'
    client_secret = '6b4958253cb544aaae1b0bf4858726ee'
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_albums_for_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    params = {"include_groups": "album"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        albums = json.loads(response.text)["items"]
        album_dict = {}

        for album in albums:
            album_id = album["id"]
            album_name = album["name"]
            album_tracks = album["total_tracks"]
            album_type = album["album_type"]
            album_release_date = album["release_date"]
            album_artists = [artist["name"] for artist in album["artists"]]
            album_images = album["images"]

            if album_name in album_dict:
                existing_tracks = album_dict[album_name]["total_tracks"]
                if album_tracks > existing_tracks:
                    album_dict[album_name] = {
                        "album_id": album_id,
                        "album_name": album_name,
                        "total_tracks": album_tracks,
                        "album_type": album_type,
                        "release_date": album_release_date,
                        "artists": album_artists,
                        "images": album_images
                    }
            else:
                album_dict[album_name] = {
                    "album_id": album_id,
                    "album_name": album_name,
                    "total_tracks": album_tracks,
                    "album_type": album_type,
                    "release_date": album_release_date,
                    "artists": album_artists,
                    "images": album_images
                }
        return album_dict
    else:
        print("Error: Failed to get albums for artist")
        print(response.content)
        return None
    
def get_tracks_for_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    params = {"market": "US"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks = json.loads(response.text)["items"]
        track_list = []
        for track in tracks:
            track_name = track["name"]
            track_id = track["id"]
            track_number = track["track_number"]
            track_explicit = track["explicit"]
            track_artists = [artist["name"] for artist in track["artists"]]
            track_dict = {
                "track_name": track_name,
                "track_id": track_id,
                "track_number": track_number,
                "track_explicit": track_explicit,
                "track_artists": track_artists,
            }
            track_list.append(track_dict)
        return track_list
    else:
        print("Error: Failed to get tracks for album")
        print(response.content)
        return None

def get_audio_features(token, track_ids):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://api.spotify.com/v1/audio-features"
    
    id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]
    
    dfs = []
    for chunk in id_chunks:
        params = {"ids": ",".join(chunk)}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()["audio_features"]
        features = pd.DataFrame(data)
        features = features.drop(["type", "uri", "track_href", "analysis_url"], axis=1)
        dfs.append(features)
    
    features = pd.concat(dfs, ignore_index=True)
    return features

In [6]:
token = get_token()
albums= get_albums_for_artist(token, '3pKJFnkhwHIK7Cvvz5K06Y')

In [173]:
albums

{}

In [165]:
album_df = pd.DataFrame.from_dict(albums, orient="index")
album_df.index.name = "album_name"

In [166]:
album_df

""
album_name


In [167]:
all_tracks_df = pd.DataFrame()
for album_id in album_df['album_id']:
    tracks = get_tracks_for_album(token, album_id)
    tracks = pd.DataFrame(tracks)
    tracks['album_id'] = album_id
    all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)

all_tracks_df

KeyError: 'album_id'

In [ ]:
af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
af_df = af_df.rename(columns={'id': 'track_id'})
af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,duration_ms,time_signature
0,0.197,0.550,8,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,6OJ2Q3WF6fWsSDXJ7x21ja,55880,3
1,0.422,0.955,2,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,1YQwSXwOyiHsDhuHK4qwef,366200,4
2,0.338,0.978,0,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,1OCJTJmvLswgUFmWGyAfMB,307667,4
3,0.469,0.933,2,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,4A8maWsDAZ51QngVy9tU4S,427680,4
4,0.378,0.981,0,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,6VOJIfQBE4P21mnBA82nIK,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.267,0.990,4,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,5MyoidtsejjnjTcwXPXBlo,71733,4
155,0.315,0.923,7,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,4ZnhD57uwrP2935k2ziLvi,206933,4
156,0.316,0.971,2,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,5ZOPIDzkfKfLHnqXvOmVlf,255267,4
157,0.452,0.962,9,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,6Kksr5Cu3eVFsVmjQ3Y1Te,186467,3


In [ ]:
all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
all_aftracks_df

,track_name,track_id,track_number,track_explicit,track_artists,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Intro - Live,6OJ2Q3WF6fWsSDXJ7x21ja,1,False,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.197,0.550,8,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,55880,3
1,Critical Acclaim - Live,1YQwSXwOyiHsDhuHK4qwef,2,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.422,0.955,2,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,366200,4
2,Second Heartbeat - Live,1OCJTJmvLswgUFmWGyAfMB,3,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.338,0.978,0,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,307667,4
3,Afterlife - Live,4A8maWsDAZ51QngVy9tU4S,4,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.469,0.933,2,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,427680,4
4,Beast and the Harlot - Live,6VOJIfQBE4P21mnBA82nIK,5,False,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.378,0.981,0,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Breaking Their Hold,5MyoidtsejjnjTcwXPXBlo,9,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.267,0.990,4,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,71733,4
155,Forgotten Faces,4ZnhD57uwrP2935k2ziLvi,10,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.315,0.923,7,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,206933,4
156,Thick And Thin,5ZOPIDzkfKfLHnqXvOmVlf,11,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.316,0.971,2,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,255267,4
157,Streets,6Kksr5Cu3eVFsVmjQ3Y1Te,12,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.452,0.962,9,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,186467,3


In [ ]:
album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
album_all_aftracks_df

,album_id,album_name,total_tracks,album_type,release_date,artists,images,track_name,track_id,track_number,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Intro - Live,6OJ2Q3WF6fWsSDXJ7x21ja,1,...,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,55880,3
1,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Critical Acclaim - Live,1YQwSXwOyiHsDhuHK4qwef,2,...,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,366200,4
2,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Second Heartbeat - Live,1OCJTJmvLswgUFmWGyAfMB,3,...,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,307667,4
3,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Afterlife - Live,4A8maWsDAZ51QngVy9tU4S,4,...,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,427680,4
4,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Beast and the Harlot - Live,6VOJIfQBE4P21mnBA82nIK,5,...,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Breaking Their Hold,5MyoidtsejjnjTcwXPXBlo,9,...,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,71733,4
155,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Forgotten Faces,4ZnhD57uwrP2935k2ziLvi,10,...,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,206933,4
156,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Thick And Thin,5ZOPIDzkfKfLHnqXvOmVlf,11,...,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,255267,4
157,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Streets,6Kksr5Cu3eVFsVmjQ3Y1Te,12,...,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,186467,3


In [ ]:
album_all_aftracks_df.columns

Index(['album_id', 'album_name', 'total_tracks', 'album_type', 'release_date',
       'artists', 'images', 'track_name', 'track_id', 'track_number',
       'track_explicit', 'track_artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [ ]:
rot = ['a', 'b', 'c', 'd', 'e', 'f']

rot_i = 0

for i in range(0, 20):
    rot_i = (rot_i + 1) % len(rot)
    print(rot[rot_i])

b
c
d
e
f
a
b
c
d
e
f
a
b
c
d
e
f
a
b
c


In [170]:
seesee = pd.read_parquet("Avenged Sevenfold_tracks.parquet")
seesee.columns

Index(['album_id', 'album_name', 'total_tracks', 'album_type', 'release_date',
       'artists', 'images', 'track_name', 'track_id', 'track_number',
       'track_explicit', 'track_artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')